# OR-Tools

For this class we will be using the Google OR-tools library to solve our Linear Programs. OR-tools is an optimization librarby made by Google that can solve many different types of optimization problems. You can find the documentation here:

https://developers.google.com/optimization

The nice thing about OR tools is that it allows you to formulate problems the same accross different programming languages. I originally started using this in C# but have move to also using it in python. It is a nice API for formulating your problems but is also separated from the solver on the backend so you can plug in different solvers if you want / need (cplex or gorubi for example).

First thing is first, let's install OR-tools. If you plan to use this through colab you need to install the or-tools dependency with the line below:

In [ ]:
!pip install ortools

     |████████████████████████████████| 32.7MB 123kB/s 


If you instead want to use it from a local python installation you would type the following line into a python terminal prompt:

python -m pip install -upgrade -user ortools

# Linear Programming Example

#Integer Programming

### x xor y, $x \oplus y = (x \cup y) - (x\cap y)$

The exclusive or is a useful constraints for things that are replacements of one another. Maybe you need to make a decision between two equivalent options as part of your package. Maybe you want exactly one piece of fruit in your lunch either an apple or a banana, but not both. That's xor.

|X|Y|$\oplus$|
|---|---|---|
|True|True|False|
|True|False|True|
|False|True|True|
|False|True|False|

Let's use the case where x and y are both binary numbers: either 0 or 1.

We can rewrite our table as:

|X|Y|$\oplus$|
|---|---|---|
|1|1|0|
|1|0|1|
|0|1|1|
|0|0|0|

If you absolutely need to have at least one of the items  then we can write:
$x+y =1$ Notice that with this constraint one of values will be 1 and the other will be zero.

|X|Y|X+Y|=1|
|---|---|---|---|
|1|1|2|False|
|1|0|1|True|
|0|1|1|True|
|0|0|0|False|


If you need one or the other or neither then you would write the below:
$x+y \leq 1$ By relaxing the equality into less than or equal, you are allowing for the set of numbers where both X and Y are zero.

|X|Y|X+Y|$\leq1$|
|---|---|---|
|1|1|2|False|
|1|0|1|True|
|0|1|1|True|
|0|0|0|True|

We can calculate xor of numbers in python using the ^ symbol. Remember this is not exponentiation, so use with caution. This can be useful for a lot of cool algorithms but most use cases are outside the scope of this course. For now just know that this tool is available as a way to debug your code or test inputs and outputs in your shell.

In [1]:
print(True^False)
print(False^True)
print(True^True)
print(False^False)
print(1^0)
print(0^1)
print(1^1)
print(0^0)

True
True
False
False
1
1
0
0


### x and y
Intersection is very useful for when things come in pairs or packages and are useless alone. Let's say that you need to buy place settings for a party or you need to equip a marching band with uniforms. You would want to pair each fork with a knife and each hat with a jacket.


Again, let's assume that x and y are binary values. If we want packages where they come in equal amoutns we would normally write $x=y$. We can rearrange this equation to be 
$x-y = 0$

If we have a third item in there, z, we would only need to introduce one more constraint: $y-z =0$. We can now daisy chain these inequality thanks to transitivity. Since $x=y$ and $y=z$ we have that $x=y=z$ without further complicating our model.

|x|y|$\cap$|x-y| =0|
|---|---|----|---|---|
|1|1|True|0|True|
|1|0|False|1|False|
|0|1|False|1|False|
|0|0|False|0|True|

#Knapsack

#Sudoku

#Travelling Salesperson Problem

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 25 19:48:36 2017

@author: tran2
"""
#!pip install pulp
from pulp import *
import matplotlib.pyplot as plt
num_cities = 6
city_locations= [(0,0),(0,.1),(.1,0),(1,1),(.9,1),(1,.9)]

#distance between city x and city y
def calc_distance(x,y):
    d= ((x[0]-y[0])**2+(x[1]-y[1])**2)**(.5)
    return d

#given 2d key, gives you the 1d equivalent
def get_key(i,j,num_cities):
    return i*num_cities+j

#given 1d key, ivest you the 2d equivalent
def get_index(key,num_cities):
    return (key//num_cities,key%num_cities)

#takes in list of tuples
def pretty_path_display(cur,optimal,pretty_optimal,num_cities):
    if len(pretty_optimal) == num_cities:
        return pretty_optimal
    else:
        for path in optimal:
            if path[0] == cur:
                pretty_optimal.append(path[1])
                return pretty_path_display(path[1],optimal,pretty_optimal,num_cities)
def plot_path(pretty_path,city_locations):
    fig,ax = plt.subplots()
    ax.plot([city_locations[city][0] for city in pretty_path],[city_locations[city][1] for city in pretty_path],marker = 'o')
    for i,loc in enumerate(city_locations):
        ax.annotate(i,loc)
distances = [[calc_distance(x,y) for y in city_locations]for x in city_locations]


tsp = LpProblem("Traveling Salesman",LpMinimize)
path_vars = LpVariable.dicts("Paths",range(num_cities**2),cat = "Binary")
dummy_vars = LpVariable.dicts("Dummies",range(num_cities))

#define objective
tsp += sum([distances[i][j]*path_vars[get_key(i,j,num_cities)] for i in range(num_cities) for j in range(num_cities)])

#departure constraints
for k in range(num_cities):
    tsp += sum([path_vars[get_key(k,j,num_cities)] 
            for j in range(num_cities)]) == 1
#arrival conditions
for k in range(num_cities):
    tsp += sum([path_vars[get_key(i,k,num_cities)] for i in range(num_cities)])==1

#no loop conditions
for i in range(1,num_cities):
    for j in range(1,num_cities):
        tsp += (dummy_vars[i] - dummy_vars[j] 
                + num_cities*path_vars[get_key(i,j,num_cities)] <= num_cities - 1)

tsp.solve()

optimal_paths = []
pretty_optimal_paths = []

for path in range(num_cities**2):
    if path_vars[path].value() == 1.0:
        optimal_paths.append(get_index(path,num_cities))
pretty_optimal_paths = pretty_path_display(0,optimal_paths,pretty_optimal_paths,num_cities)
plot_path(pretty_optimal_paths,city_locations)
print("The Optimal Path is:\t",end = "")
print(pretty_optimal_paths)
total_distance = 0

for index in optimal_paths:
    total_distance += distances[index[0]][index[1]]
    
print("The total distance is :\t {:.3f}".format(total_distance))

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Oct 26 09:45:44 2017

@author: tran260
"""

from pulp import *
num_cities = 6
#distance between city x and city y
def calc_distance(x,y):
    d= ((x[0]-y[0])**2+(x[1]-y[1])**2)**(.5)
    return d
city_locations= [(0,0),(0,.1),(.1,0),(1,1),(.9,1),(1,.9)]
distances = [[calc_distance(x,y) for y in city_locations]for x in city_locations]

#returns a binary variable
#starts at the first city of optimal_path[0][0]
#append every node that contains either the city or a decendant
def is_sub_path(first_city,optimal_path,num_cities):
    if first_city == -1:
        return False
    tour = path_tour(first_city)
    if tour == num_cities:
        return False
    else:
        return True

def path_tour(cur_city,optimal_path,tour):
    for path in optimal_path:
        if cur_city == path[0] and path not in tour:
            tour.append(path)
            tour = path_tour(path[1],optimal_path,tour)
        elif cur_city == path[1] and path not in tour:
            tour.append(path)
            tour = path_tour(path[0],optimal_path,tour)
    return tour

"""Need To modify!!!"""
def pretty_path_display(cur,optimal,pretty_optimal,num_cities):
    if len(pretty_optimal) == num_cities:
        return pretty_optimal
    else:
        for path in optimal:
            if path[0] == cur:
                pretty_optimal.append(path[1])
                return pretty_path_display(path[1],optimal,pretty_optimal,num_cities)
while(True):
    tsp = LpProblem("Traveling Salesman",LpMinimize)
    path_vars = LpVariable.dicts("Paths",(range(num_cities),range(num_cities)),cat = "Binary")
    #define objective
    tsp += sum([distances[i][j]*path_vars[i][j] for i in range(num_cities) for j in range(num_cities)])
    
    for i in range(num_cities):
        for j in range(num_cities):
            if i>=j:
                tsp += path_vars[i][j] == 0
    
    for i in range(num_cities):
        tsp += sum([path_vars[i][j] for j in range(i,num_cities)])+sum([path_vars[j][i] for j in range(i)]) == 2
    
    tsp.solve()
    
    optimal_path = []
    for i in range(num_cities):
        for j in range(num_cities):
            if path_vars[i][j].value() == 1.0:
                optimal_path.append([i,j])
    first_city = optimal_path[0][0]
    tour = path_tour(first_city,optimal_path,[])

    if len(tour) != num_cities :
        print("continuing")
        print(tour)
        pretty_tour = pretty_path_display(tour[0][0],tour,[],len(tour))
        print(pretty_tour)
        tsp += sum([path_vars[i][j] for i in pretty_tour for j in pretty_tour])<=len(tour) - 1
    else:
        print(len(tour)==num_cities)
        print(optimal_path)
        print(tour)
        break